In [2]:
import pandas as pd
import polars as pl

Cleaning strategy for NPPES file:
- read in only columns relevant to this project
- process taxonomy code columns to determine primary
- drop taxonomy code columns (keep only added primary code column)
- make column names better for SQL
- join classification crosswalk to get taxonomy name
- join CBSA crosswalk to match provider on zip code

In [64]:
# generate list of taxonomy columns
tax_cols = []

for i in range(1, 16):
    switch_col_name = f'Healthcare Provider Primary Taxonomy Switch_{i}'
    tax_cols.append(switch_col_name)
    code_col_name = f'Healthcare Provider Taxonomy Code_{i}'
    tax_cols.append(code_col_name)

# other relevant columns
cols = ['NPI', 'Entity Type Code', 'Provider Organization Name (Legal Business Name)', 'Provider Last Name (Legal Name)',
        'Provider First Name', 'Provider Middle Name', 'Provider Name Prefix Text', 'Provider Name Suffix Text',
        'Provider Credential Text', 'Provider First Line Business Practice Location Address',
        'Provider Second Line Business Practice Location Address', 'Provider Business Practice Location Address City Name',
        'Provider Business Practice Location Address State Name', 'Provider Business Practice Location Address Postal Code']

# combine column name lists
cols.extend(tax_cols)

In [71]:
npi = pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', usecols = cols)

/var/folders/xm/vmtk5pms0jzb2kmtm00lwtn80000gn/T/ipykernel_36556/1476842930.py:1: DtypeWarning: Columns (32,67,70,71,74,75,78,79,82,83,86,87,90,91,94,95,98,99,102,103,106) have mixed types. Specify dtype option on import or set low_memory=False.
  npi = pd.read_csv('../data/npidata_pfile_20050523-20240211.csv', usecols = cols)


In [66]:
# modify read in to get necessary columns and rename at the same time
npi.head()

,NPI,Entity Type Code,Provider Organization Name (Legal Business Name),Provider Last Name (Legal Name),Provider First Name,Provider Middle Name,Provider Name Prefix Text,Provider Name Suffix Text,Provider Credential Text,Provider First Line Business Practice Location Address,...,Healthcare Provider Taxonomy Code_11,Healthcare Provider Primary Taxonomy Switch_11,Healthcare Provider Taxonomy Code_12,Healthcare Provider Primary Taxonomy Switch_12,Healthcare Provider Taxonomy Code_13,Healthcare Provider Primary Taxonomy Switch_13,Healthcare Provider Taxonomy Code_14,Healthcare Provider Primary Taxonomy Switch_14,Healthcare Provider Taxonomy Code_15,Healthcare Provider Primary Taxonomy Switch_15
0,1679576722,1.0,NaN,WIEBE,DAVID,A,NaN,NaN,M.D.,3500 CENTRAL AVE,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1588667638,1.0,NaN,PILCHER,WILLIAM,C,DR.,NaN,MD,1824 KING STREET,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1497758544,2.0,"CUMBERLAND COUNTY HOSPITAL SYSTEM, INC",NaN,NaN,NaN,NaN,NaN,NaN,3418 VILLAGE DR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1306849450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1215930367,1.0,NaN,GRESSOT,LAURENT,NaN,DR.,NaN,M.D.,17323 RED OAK DR,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [73]:
# pull primary taxonomy codes into one column
def extract_taxonomy_code(row):

    for i in range(1, 16):
        switch_col_name = f'Healthcare Provider Primary Taxonomy Switch_{i}'
        if row[switch_col_name] == 'Y':
            code_col_name = f'Healthcare Provider Taxonomy Code_{i}'
            return row[code_col_name]
    
    return None # catch columns with no 'Y' switch

In [74]:
npi['extracted_taxonomy_code'] = npi.apply(extract_taxonomy_code, axis = 1)

In [75]:
npi.info()
# having doublechecked the values in the switch columns, it appears that there are rows with no 'Y' switch
# should those rows be dropped, or possibly the taxonomy code in Code_1 could be inferred to be the correct code?
# some rows with no 'Y' switch have one or more 'X' switches

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8180697 entries, 0 to 8180696
Data columns (total 45 columns):
 #   Column                                                   Dtype  
---  ------                                                   -----  
 0   NPI                                                      int64  
 1   Entity Type Code                                         float64
 2   Provider Organization Name (Legal Business Name)         object 
 3   Provider Last Name (Legal Name)                          object 
 4   Provider First Name                                      object 
 5   Provider Middle Name                                     object 
 6   Provider Name Prefix Text                                object 
 7   Provider Name Suffix Text                                object 
 8   Provider Credential Text                                 object 
 9   Provider First Line Business Practice Location Address   object 
 10  Provider Second Line Business Practice Loc

In [48]:
# if we decide to drop rows with no 'Y' switch
# npi.dropna(subset = ['extracted_taxonomy_code'], inplace = True)

In [76]:
# convert column names to better format

npi.columns = [x.lower().replace(' ', '_') for x in npi.columns]

In [77]:
npi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8180697 entries, 0 to 8180696
Data columns (total 45 columns):
 #   Column                                                   Dtype  
---  ------                                                   -----  
 0   npi                                                      int64  
 1   entity_type_code                                         float64
 2   provider_organization_name_(legal_business_name)         object 
 3   provider_last_name_(legal_name)                          object 
 4   provider_first_name                                      object 
 5   provider_middle_name                                     object 
 6   provider_name_prefix_text                                object 
 7   provider_name_suffix_text                                object 
 8   provider_credential_text                                 object 
 9   provider_first_line_business_practice_location_address   object 
 10  provider_second_line_business_practice_loc